## Initialize Input Files for IPMOF (initialize_IPMOF.py)

### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cutOff radius and grid size for energy map

### Create input files for MOF files in a given directory
1. Reaf MOF files in ".mol2" format from given directory and create a list
2. Choose the first MOF1 as the stationary (map) MOF, initialize its variables, and calculate packed coordinates
3. For all the MOFs in the list:
4. Initialize MOF2 variables for the mobile MOF
5. Calculate energy map
6. Export energy map -> .json
7. Export MOF2 structure information -> .json

In [1]:
import math
import os
#os.chdir('C:\Kutay\IPMOF')
from energyMapFunctions import *

# Initialize force field parameters
excelFileDir = 'C:\\Users\\kutay\\iPython\\IPMOF\\FF_Parameters.xlsx'
UFF = readFFparameters(excelFileDir, 'UFF')
cutOff = 13

# Generate MOF list
fileDir = 'C:\\Kutay\\MOFs'
fileFormat = ".mol2"
MOFlist = generateMOFlist(fileDir, fileFormat)
print(MOFlist)

MOFs = []
i = 0
for mof in MOFlist:
    MOFs.append(MOF())
    MOFs[i].dir = fileDir + '\\' + mof
    MOFs[i].file = open(MOFs[i].dir, 'r')
    MOFs[i].initialize()
    MOFs[i].initializeFF(UFF)
    i += 1

['AgH6_CN2_3.mol2', 'Cd2H26C17_NO2_6.mol2', 'Cd2HC9NO8.mol2', 'Co3H16_C9O7_2.mol2', 'CuNiH8C12_N2O_4.mol2', 'MgH2_CO2_2.mol2', 'MOF5.mol2', 'NiH6_C7O3_2.mol2', 'YbH17C30_NO3_2.mol2', 'Zn2H22C27_NO_8.mol2', 'Zn4H12C24O13.mol2', 'Zn7_PO4_6.mol2']


In [2]:
for MOFindex in range(len(MOFs)):   
    # Calculate packing for stationary MOF
    packingFactor = Packing.factor(MOFs[MOFindex].UCsize, cutOff)
    print('Packing factor for ' + MOFlist[MOFindex].split('.')[0] + ' calculated as: ' + str(packingFactor))
    translationVectors, UCvectors = Packing.vectors(packingFactor, MOFs[MOFindex].UCsize, MOFs[MOFindex].UCangle)
    MOFs[MOFindex].packedCoor = Packing.UC(translationVectors, packingFactor, UCvectors, MOFs[MOFindex].atomCoor)

Packing factor for AgH6_CN2_3 calculated as: [5, 5, 5]
Packing factor for Cd2H26C17_NO2_6 calculated as: [5, 5, 3]
Packing factor for Cd2HC9NO8 calculated as: [5, 5, 5]
Packing factor for Co3H16_C9O7_2 calculated as: [5, 5, 5]
Packing factor for CuNiH8C12_N2O_4 calculated as: [5, 5, 5]
Packing factor for MgH2_CO2_2 calculated as: [5, 3, 3]
Packing factor for MOF5 calculated as: [3, 3, 3]
Packing factor for NiH6_C7O3_2 calculated as: [5, 3, 3]
Packing factor for YbH17C30_NO3_2 calculated as: [5, 5, 5]
Packing factor for Zn2H22C27_NO_8 calculated as: [5, 3, 3]
Packing factor for Zn4H12C24O13 calculated as: [3, 3, 3]
Packing factor for Zn7_PO4_6 calculated as: [5, 5, 5]


In [3]:
for MOFindex in range(len(MOFs)):  
    atomList = setUniqueAtomList(MOFs)
    # Calculate Energy Map
    eMap = energyMap(MOFs[MOFindex], atomList, cutOff, 1)
    # Create export directory for the energy map
    MOFname = MOFlist[MOFindex].split('.')[0]
    exportFolder = "C:\\Users\\kutay\\iPython\\IPMOF\\initializeIPMOF"
    exportFormat = ".js"
    exportDir_eMap = os.path.join(exportFolder, MOFname + '_eMap' + exportFormat)
    exportDir_MOF = os.path.join(exportFolder, MOFname + exportFormat)
    print(exportDir_eMap)
    print(exportDir_MOF)

    exportEnergyMapjs(eMap, atomList, exportDir_eMap)

    exportMOFjs(MOFs[MOFindex], exportDir_MOF)

C:\Users\kutay\iPython\IPMOF\initializeIPMOF\AgH6_CN2_3_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\AgH6_CN2_3.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Cd2H26C17_NO2_6_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Cd2H26C17_NO2_6.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Cd2HC9NO8_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Cd2HC9NO8.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Co3H16_C9O7_2_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\Co3H16_C9O7_2.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\CuNiH8C12_N2O_4_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\CuNiH8C12_N2O_4.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\MgH2_CO2_2_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\MgH2_CO2_2.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\MOF5_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\MOF5.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\NiH6_C7O3_2_eMap.js
C:\Users\kutay\iPython\IPMOF\initializeIPMOF\NiH6_C7O3_2.js
C:

In [2]:
atomList = setUniqueAtomList(MOFs)

eMapAtomList = getEnergyMapAtomList(MOFs, atomList)

#exportCombinationEnergyMapjs(eMap, eMapAtomList, MOFindex, exportDir)
    

In [3]:
# Calculate Energy Map
# eMap = energyMap(MOF2, atomList, cutOff, 1)

#plotEnergyMap(eMap, 90, 0)
#plotPackedCell(MOF2.packedCoor, 90, 0)

# Create export directory for the energy map
#MOFname = MOFlist[MOFselection].split('.')[0]
#MOFname = 'MOFX'
#exportFolder = "C:\\Users\\kutay\\iPython\\IPMOF\\multipleMOF"
#exportFormat = ".js"
#exportDir_eMap = os.path.join(exportFolder, MOFname + '_eMap' + exportFormat)
#exportDir_MOF = os.path.join(exportFolder, MOFname + exportFormat)
#print(exportDir_eMap)
#print(exportDir_MOF)

#eMapDir = "C:\\Users\\kutay\\iPython\\IPMOF\\eMap.js"
#exportEnergyMapjs(eMap, atomList, exportDir_eMap)

#MOFexportDir = "C:\\Users\\kutay\\iPython\\IPMOF\\MOF.js"
#exportMOFjs(MOF2, exportDir_MOF)

eMapAtomList

[{'eMapAtomIndex': [1, 3, 9, 11],
  'epsilon': [22.156453860650792,
   18.12800770416883,
   52.873355803825746,
   34.74534809965693],
  'name': ['H', 'Ag', 'C', 'N'],
  'sigma': [2.5711337005530193,
   2.8045491647057883,
   3.4308509635584463,
   3.260689308393642]},
 {'eMapAtomIndex': [0, 1, 2, 9, 11],
  'epsilon': [30.213346173614717,
   22.156453860650792,
   114.81071545973595,
   52.873355803825746,
   34.74534809965693],
  'name': ['O', 'H', 'Cd', 'C', 'N'],
  'sigma': [3.1181455134911875,
   2.5711337005530193,
   2.537279549263686,
   3.4308509635584463,
   3.260689308393642]},
 {'eMapAtomIndex': [0, 1, 2, 9, 11],
  'epsilon': [30.213346173614717,
   22.156453860650792,
   114.81071545973595,
   52.873355803825746,
   34.74534809965693],
  'name': ['O', 'H', 'Cd', 'C', 'N'],
  'sigma': [3.1181455134911875,
   2.5711337005530193,
   2.537279549263686,
   3.4308509635584463,
   3.260689308393642]},
 {'eMapAtomIndex': [0, 1, 9, 10],
  'epsilon': [30.213346173614717,
   22.15645

In [41]:
# Sort energy map according to energy values
sortedMap = sorted(eMap, key=lambda x: x[3], reverse=True)
energyLimit = sortedMap[len(MOF2.atomCoor)][3]